In [16]:
import pandas as pd

# Load the CSV files
X = pd.read_csv('./X_Features_concat_final.csv')
y = pd.read_csv('./y_open_0905~_1000(5min)/y_open_0930.csv')

# 시간 형태로 변경
X['date'] = pd.to_datetime(X['date'] ,format="%Y-%m-%d")
y['date'] = pd.to_datetime(y['date'],format= "%Y%m%d")

df = pd.merge(X,y, on='date',how='inner')
df.sort_index(ascending=False,inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)
df.set_index('date',inplace=True)

df['open'] = df['open'].shift(-1)
df['kodex_open'] = df['kodex_open'].shift(-1)
df.dropna(inplace=True)

In [17]:
df1 =df[["open","kodex_open",
"sp500_closing",
"PER",
"KR_10Y",
"kodex_volume",
"Dividend_Yield"]]

In [18]:
# X, y 정의
y = df1[['open']]
X = df1.drop(columns='open')

In [19]:
# Check the data types of the features in 'X'
X_dtypes = X.dtypes

# Display the data types that are not numeric
non_numeric_columns = X_dtypes[X_dtypes != 'float64'][X_dtypes != 'int64']
non_numeric_columns

# Function to convert percentage strings to float values
def convert_percentage(val):
    new_val = val.replace('%', '')
    return float(new_val) / 100

# Remove commas and convert percentage strings to floats
for column in non_numeric_columns.index:
    if 'percentage' in column:
        X[column] = X[column].apply(convert_percentage)
    else:
        X[column] = X[column].str.replace(',', '').astype(float)

# Confirm the conversions by checking datatypes again
X_dtypes_converted = X.dtypes
X_dtypes_converted


kodex_open        float64
sp500_closing     float64
PER               float64
KR_10Y            float64
kodex_volume        int64
Dividend_Yield    float64
dtype: object

In [23]:
X_corr=X.corr()

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a color map based on the correlation values
colors = X_corr.apply(lambda x: 'green' if x > 0 else 'red')

# Plot the horizontal bar graph
plt.figure(figsize=(10, 6))
X_corr.plot(kind='barh', color=colors)
plt.title('Correlation of Features with Open')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [26]:
X.to_csv('./real_final_feature.csv')